# **CHAPTER 3: Looking Inside Large Language Models**

In [1]:
!pip install -q "transformers==4.43.3" "accelerate>=0.31.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 70.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 75.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import transformers, torch
print("transformers =", transformers.__version__)
print("torch =", torch.__version__)

transformers = 4.43.3
torch = 2.6.0+cu124


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

name_model = "microsoft/Phi-3-mini-4k-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(name_model)

model = AutoModelForCausalLM.from_pretrained(
    name_model,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

2025-08-22 14:18:16.409736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755872296.759716      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755872296.860427      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=120,
    do_sample=False,
)

# **An Overview of Transformer Models**
Let's begin our exploration with a high-level overview of the model, and then we'll see how later work has improved upon the Transformers model since its introduction in 2017.

### **The Inputs and Outputs of a Trained Transformer LLM**
When Transformer LLMs are given a prompt, they won't generate the text in one operation; it actually generates **one token** at a time. The picture below shows four steps of token generation in response to the input prompt. Each token generation step is one forward pass through the model (that's machine-learning speak for the inputs going into the neural network and flowing through the computations it needs to produce an output on the other end of the computation graph).

> So that it's called **autogressive** models

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/four_steps_transfomers.jpg" style="display:block; margin:0 auto">

In [5]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


{'generated_text': " Mention the steps you're taking to prevent it in the future.\n\nDear Sarah,\n\nI hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden. It was a tragic mishap that I never intended, and I am truly sorry for any distress it may have caused you.\n\nThe incident happened when I was attempting to help you with your gardening project. I had been researching various techniques to improve the health and growth of your plants, and I thought I"}


In [6]:
print(output[0]['generated_text'])

 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden. It was a tragic mishap that I never intended, and I am truly sorry for any distress it may have caused you.

The incident happened when I was attempting to help you with your gardening project. I had been researching various techniques to improve the health and growth of your plants, and I thought I


### **The Components of the Forward Pass**
Two key internal components are the **tokenizer** and **the language modelling head (LM head)**. You can see where these components lie in the system. 

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/inside_transformers.jpg" style="display:block; margin: 0 auto;">

The tokenizer is followed by the **neural network**: a stack of **Transformer** blocks that do all of the processing. That stack is then followed by the **LM head**, which translates the output of the stack into **probability scores** for what the most likely next token is. 

The **LM Head** is a simple neural network layer itself. It is one of multiple possible **"heads"** to attach to a stack of Transformer blocks to build different kinds of systems. Other kinds of Transformers heads include sequence classification heads and token classification heads.

In [7]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

As we can see in this structure, the **Phi3Model** contains:
- This shows us the various nested layers of the model. The majority of the model is labelled **model** and **lm_head**.
- Inside the **Phi3Model** model, we can see the **embeddings matrix** - *embed_tokens* and its dimensions. It had **32.064 tokens** and each with a vector size of **3.072**
- The next major component is the stack of **Transformer** decoder layers. It contains **32** blocks of type **Phi3DecoderLayer**.
- Each of these **Transformer blocks** includes an **attention layer** and a **feedforward neural network** (a.k.a an **mlp** or **multilevel perceptron**).
- Finally, we see the **lm_head** taking a vector of size **3.072** and outputting a vector equivalent to the number of tokens the model knows. The output is the probability score for each token that helps us select the output token.

### **Choosing a Single Token from the Probability**
At the end of processing, the model outputs a probability score for each token in the vocabulary. Choosing a single token from the probability distribution is called ***the decoding strategy***. The easiest decoding strategy would be to always pick the token with the highest probability score. In practice, however, this doesn't tend to lead to the best outputs for most use cases. A better approach is to add some **randomness** and sometimes choose the second or third highest probability token. The idea here is to basically **sample** from the probability distribution based on the probability score, as the statisticians would say.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/prob_tokens.jpg" style="display:block; margin:0 auto;">

For example, in this case, the token "Dear" has a **40%** probability of being the next token, then it has a 40% chance of being picked. So with this method, all the other tokens have a chance of being picked according to their score.
Choosing the highest-scoring token every time is called **greedy decoding**. It's what happens if you set the **temperature** parameter to zero in an LLM.

Looking more closely at the code that demonstrates this process. 

In [8]:
prompt = "The biggest city in Vietnam is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

model_output = model.model(input_ids)

lm_head_output = model.lm_head(model_output[0])
print(lm_head_output.shape)
print(lm_head_output)

token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

torch.Size([1, 6, 32064])
tensor([[[24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
         [34.2500, 35.2500, 35.2500,  ..., 30.1250, 30.1250, 30.1250],
         [33.5000, 32.0000, 34.7500,  ..., 29.6250, 29.6250, 29.6250],
         [34.2500, 30.0000, 31.3750,  ..., 27.0000, 27.0000, 27.0000],
         [34.5000, 33.2500, 36.2500,  ..., 29.0000, 29.0000, 29.0000],
         [28.7500, 27.2500, 27.2500,  ..., 19.3750, 19.3750, 19.3750]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)


'Ho'

### **Speeding up Generation by Caching Keys and Values**

If we give the model the ability to cache the results of the previous calculation (especially some of the specific vectors in the attention mechanism), we no longer need to repeat the calculations of the previous streams. This time, the only needed calculation is for the last stream. This is an optimization technique called the **keys and values (KV) cache**, and it provides a significant speedup of the generation process. 

In [11]:
%%timeit -n 1
#* NOT USING CACHE
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")


generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=False
)

32.9 s ± 1.05 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit -n 1
#* NOT USING CACHE
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")


generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=True
)

4.55 s ± 42.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### **Inside the Transformer Block**

As the picture shows, **Transformer LLMs** are composed of a series of Transformer blocks (often in the range of six in the original Transformer paper, to over a hundred in many large LLMs). Each block processes its input, then passes the results of its processing to the next block.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/inside_transformer_block.jpg" style="display:block; margin: 0 auto;">

The Transformer block is made up of two successive components:
- **The attention layer** is mainly concerned with incorporating.
- **The feedforward layer** houses the majority of the model's processing capacity.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/transformer_blocks_components.jpg" style="display:block; margin: 0 auto;">

## **Attention is All you need**
**Attention** is a mechanism that helps the model incorporate context as it's processing a specific token.
> The dog chased the squirrel because **it**

For the model to predict what comes after "it", it needs to know what "it" refers to. 

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/attention_image.jpg" style="display:block; margin:0 auto">

The picture shows multiple **token positions** going into the attention layer; the final one is the one being currently processed (the pink arrow). The attention mechanism operates on the input vector at that position. It incorporates relevant information from the context into the vector it produces as the output for that position.

Two main steps are involved in the attention mechanism:
- A way to score how relevant each of the previous input tokens are to the current token being processed (in the pink arrow)
- Using those scores, we combine the information from the various positions into a single output vector.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/attention_two_steps.jpg" style="display:block; margin:0 auto">

## **How attention is calculated**

Observe the following as the starting position:

- The **attention** layer (of a generative LLM) is processing attention for a single position.
- The inputs to the layer are:
    + The vector representation of the current position or token
    + The vector representations of the previous tokens
- The goal is to produce a new representation of the current position that incorporates relevant information from the previous tokens:
    + For example, the sentence **"Sarah fed the cat because it"**, we want **"it"** to represent the cat - so attention bakes in **"cat information"** from the cat token
- The training process produces three **projection matrices** that produce the components that interact in this calculation:
    + A **query** projection matrix
    + A **key** projection matrix
    + A **value** projection matrix
<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/attention_head_how_it_calculated.jpg" style="display:block; margin:0 auto">

## **Self-attention: Relevance scoring**

The attention mechanism here is only concerned with this one position and how information from other positions can be pulled in to inform this position.

The relevance scoring step of attention is conducted by **multiplying the query vector** of the current position with **the keys matrix**. This produces a **score stating** how relevant each previous token is. Passing that by a **softmax** operation normalizes these scores so they sum up to 1. 

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/attention_result.jpg" style="display:block; margin:0 auto">

## **Self-attention: Combining information**

After we have the relevance scores, we multiply the **value vector** associated with each token by that token's score. Summing up those resulting vectors produces the output of this attention step.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/final_attention_result.jpg" style="display:block; margin:0 auto">

# **The Transformer Block**

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/detail_transformer_block.jpg" style="display:block; margin:0 auto">

One of the differences we see in this version of the Transformer block is that **normalization** happens prior to attention and the feedforward layers. This has been reported to reduce the required training time. Another improvement in normalization here is using **RMSNorm**, which is simpler and more efficient than the **LayerNorm** used in the original Transformer. Lastly, instead of the original Transformer's **ReLU activation function**, newer variants like **SwiGLU** are now more common.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/2024_era_transformer_block.jpg" style="display:block; margin:0 auto">

## **Positional Embeddings (RoPE)**

**Positional Embeddings** have been a key component since the original Transformer. They enable the model to keep track of the order of **tokens/words** in a sequence/sentence, which is an **indispensable** source of information in language. From the many positional encoding schemes proposed in the past years, **rotary positional embeddings** (or **RoPE**
introduced in <a href="https://arxiv.org/abs/2104.09864v4" style="color:#8e0012">**“RoFormer: Enhanced Transformer with rotary position
embedding”**</a>) 

The original Transformer paper and some of the early variants had absolute positional embedding that, in essence, marked the first token as **position 1**, the second as **position 2** ... et. But some challenges arise from such methods when we scale up models, which requires us to find ways to improve their efficiency.

For example, one challenge in efficiently training models with **large context** is that a lot of documents in the training set are much shorter than that context. It would be inefficient to allocate the entire, say, **4K context** to a short **10-word sentence**. So during model training, documents are packed together into each context in the training batch.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/packed_documents.jpg" style="display:block; margin:0 auto">

**Positional embedding** methods have to adapt to this and other practical considerations. If **Document 50**, for example, stars at position 50, then we'd be **misinforming** the model if we tell it that the first token is number 50, and that would affect its performance **(because it would assume there's preivous context while in reality the earlier tokens belong to a different and unrelated document the model should ignore)**

Instead of static, absolute embeddings that are added at the beginning of the forward pass, rotary embeddings are a method to encode positional information in a way that captures absolute and relative token position information.

<img src="https://raw.githubusercontent.com/khang1108/learning_ai/refs/heads/main/image/rotary_embeddings_position.jpg" style="display:block; margin:0 auto">